In [1]:
import os
import torch

from PIL import Image

# Model Load

In [5]:
dict_path = "../results/all_in_moda_test_v0.1/checkpoint.pth"

In [6]:
checkpoint_dict = torch.load(dict_path)

In [7]:
checkpoint_dict.keys()

dict_keys(['student', 'teacher', 'optimizer', 'epoch', 'args', 'dino_loss', 'fp16_scaler'])

In [ ]:
python3 eval_knn.py \
--arch vit_tiny \
--data_path data/all_in_moda \
--pretrained_weights results/all_in_moda_test_v0.1/checkpoint.pth \
--use_cuda 